In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('stocks.csv')
print("Dataset Preview:")
print(data.head())

print("\nData Types:")
print(data.dtypes)


Dataset Preview:
  Ticker        Date        Open        High         Low       Close  \
0   AAPL  2023-02-07  150.639999  155.229996  150.639999  154.649994   
1   AAPL  2023-02-08  153.880005  154.580002  151.169998  151.919998   
2   AAPL  2023-02-09  153.779999  154.330002  150.419998  150.869995   
3   AAPL  2023-02-10  149.460007  151.339996  149.220001  151.009995   
4   AAPL  2023-02-13  150.949997  154.259995  150.919998  153.850006   

    Adj Close    Volume  
0  154.414230  83322600  
1  151.688400  64120100  
2  150.639999  56007100  
3  151.009995  57450700  
4  153.850006  62199000  

Data Types:
Ticker        object
Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


In [13]:
X = data.drop(columns=['Volume'])  # Drop the target column
y = data['Volume']  # Target variable

# Ensure all non-numeric columns are encoded 
X = pd.get_dummies(X, drop_first=True)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
import torch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [20]:
# Import necessary PyTorch modules 
import torch
import torch.nn as nn
import torch.optim as optim

# Defined the StockNeuralNetwork class with layers and activation functions
class StockNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(StockNeuralNetwork, self).__init__()
        self.first_layer = nn.Linear(input_size, hidden_sizes[0]) # First layer
        self.relu1 = nn.ReLU()  # Activation function
        self.second_layer = nn.Linear(hidden_sizes[0], hidden_sizes[1]) # Second layer
        self.relu2 = nn.ReLU() # Activation function
        self.dropout2 = nn.Dropout(0.5)  # dropout for regularization
        self.output_layer = nn.Linear(hidden_sizes[1], output_size) # output layer


# Defined the forward pass logic
    def forward(self, x):
        x = self.first_layer(x)
        x = self.relu1(x)  
        x = self.second_layer(x)
        x = self.relu2(x)  
        x = self.dropout2(x)  
        x = self.output_layer(x)
        return x

In [ ]:
# Set the model parameters(Still testing to see which are the best parameters to use)
"""Model Parameters
input_size =  
hidden_sizes =  
output_size = 



# Instantiate the neural network model
model = StockNeuralNetwork(input_size, hidden_sizes, output_size)

# Defined the loss function and optimizer
criterion = nn.MSELoss()  #
optimizer = optim.Adam(model.parameters(), lr=0.001)

"""

In [ ]:
#Improved Neural Network with Batch normalization and an additional layer
class ImprovedStockNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(ImprovedStockNeuralNetwork, self).__init__()
        self.first_layer = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])  # Batch normalization for the first layer
        self.relu1 = nn.ReLU()
        self.second_layer = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])  # Batch normalization for the second layer
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)  # Dropout after the second layer
        # Third hidden layer (newly added)
        self.third_layer = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])  # Batch normalization for the third layer
        self.relu3 = nn.ReLU()

        # Output layer
        self.output_layer = nn.Linear(hidden_sizes[2], output_size)

    def forward(self, x):
        x = self.first_layer(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.second_layer(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        # Third layer(newly added)
        x = self.third_layer(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.output_layer(x)
        return x


In [ ]:
# Set the parameters of the improved model(parameters will be added later)
"""
# Model Parameters
input_size =  
hidden_sizes =  
output_size =  
"""

In [ ]:
import random
from sklearn.metrics import mean_squared_error

# Define the hyperparameter space
hyperparameter_space = {
    "learning_rate": [],  # Add learning rates to test
    "hidden_sizes": [],  # Add configurations of hidden layers
    "dropout_rate": [],  # Add dropout rates to test
    "batch_size": [],  # Add batch sizes to test
    "epochs": []  # Add number of epochs to test
}



In [ ]:
# Random search framework
def random_search(X_train, y_train, X_test, y_test, num_trials):
    best_model = None
    lowest_loss = float("inf")

    for _ in range(num_trials):
        # Sample hyperparameters
        lr = random.choice(hyperparameter_space["learning_rate"])
        hidden_sizes = random.choice(hyperparameter_space["hidden_sizes"])
        dropout_rate = random.choice(hyperparameter_space["dropout_rate"])
        batch_size = random.choice(hyperparameter_space["batch_size"])
        epochs = random.choice(hyperparameter_space["epochs"])

        # Define model
        model = ImprovedStockNeuralNetwork(input_size, hidden_sizes, output_size)
        model.dropout2.p = dropout_rate

        # Define optimizer and loss function
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.MSELoss()

        # Train model (example logic, training details will be finalized later)
        model.train()
        for epoch in range(epochs):
            for i in range(0, X_train.size(0), batch_size):
                X_batch = X_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                optimizer.zero_grad()
                predictions = model(X_batch)
                loss = criterion(predictions, y_batch)
                loss.backward()
                optimizer.step()

        # Evaluate model
        model.eval()
        with torch.no_grad():
            predictions = model(X_test)
            val_loss = mean_squared_error(y_test.numpy(), predictions.numpy())

        # Track best model
        if val_loss < lowest_loss:
            lowest_loss = val_loss
            best_model = model

    return best_model, lowest_loss

